# Planet Property Calculator - SINGLE DIP

In [ ]:
import requests
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import lightkurve as lk
import math
from astropy import units as u
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output
tic = widgets.Text(
       value='',
       description='TIC:', )
menu_author = widgets.Dropdown(
       options=['SPOC', '*SPOC*', 'K2', '*'],
       value='*',
       description='Author:')
menu_exptime = widgets.Dropdown(
       options=['*', '120', '20', '1800'],
       value='*',
       description='Exptime:')

button_run = widgets.Button(description='Run Query', button_style='primary')
out_run = widgets.Output()
def on_run_button_clicked(_):
    global available_data_select
    global TIC_no
    global TIC
    global sectors
    with out_run:
        clear_output()
        author = menu_author.value
        if menu_exptime.value != '*':
            exptime = int(menu_exptime.value)
        else:
            exptime = menu_exptime.value
        TIC_no = tic.value.strip()
        TIC = 'TIC ' + TIC_no
        print("Searching for TIC = {} Author = {} Exptime = {}".format(TIC,author,exptime))
        # Retrieve the list of available sectors for this TIC
        available_data_select = lk.search_lightcurve(TIC, author=author, exptime=exptime)
        print(available_data_select)
        sectors = []
        for x in available_data_select.mission:
            sectors.append(x)
        sectors=np.unique(np.char.replace(sectors,"TESS Sector ",""))
        print("Sectors {}".format(sectors))          
# linking button and function together using a button's method
button_run.on_click(on_run_button_clicked)

box = widgets.VBox([tic,menu_author,menu_exptime,button_run,out_run ])
box

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output
menu_sectors = widgets.Dropdown(
       options=sectors,
       value=sectors[0],
       description='Sector:')
menu_fauthor = widgets.Dropdown(
       options=['SPOC', '*SPOC*', 'K2', '*'],
       value='SPOC',
       description='Author:')
menu_fexptime = widgets.Dropdown(
       options=['*', '120', '20', '1800'],
       value='120',
       description='Exptime:')

button_fetch = widgets.Button(description='Fetch Data', button_style='primary')
out_fetch = widgets.Output()
def on_fetch_button_clicked(_):
    global lc_collection
    global sector_no
    global exptime
    with out_fetch:
        clear_output()
        sector_no = int(menu_sectors.value)
        author = menu_fauthor.value
        exptime = int(menu_fexptime.value)
        print("Getting data for Sector = {} Author = {} Exptime = {}".format(sector_no,author,exptime))
        sector_data = lk.search_lightcurve(TIC,sector=sector_no,author=author,exptime=exptime)
        lc_collection = sector_data.download()
        print(lc_collection)
# linking button and function together using a button's method
button_fetch.on_click(on_fetch_button_clicked)

box1 = widgets.VBox([menu_sectors,menu_fauthor,menu_fexptime,button_fetch,out_fetch ])
box1

# Plot the lightcurve

In [ ]:
# First with binned data overlay
lc_collection = lc_collection.normalize()
bin_time = 10/24/60
lc_binned = lc_collection.bin(bin_time)

In [ ]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
printmd("**Right-click the six parameters of the dip** -  1: Start   2: End    3: Start of floor    4: End of floor    5: Top    6: Bottom")

%matplotlib widget
fig, ax = plt.subplots(figsize=(16,8))
lc_collection.plot(ax=ax, linewidth=0,marker='o', color='gold',markersize=1,label="Flux")
lc_binned.plot(ax=ax, linewidth=0,marker='o', color='black',markersize=3, label="Binned")
#lc_collection.plot(ax=ax, column='sap_flux', normalize=True, label='SAP Flux', linewidth=0,marker='o', color='orange',markersize=1)
#lc_binned.plot(ax=ax, column='sap_flux', normalize=True, linewidth=0,marker='o', color='black',markersize=1)

fluxmin = np.nanmin(lc_collection['flux'])
fluxmax = np.nanmax(lc_collection['flux'])
ymax = fluxmax+(fluxmax-1)*0.1
ymin = fluxmin-(fluxmax-1)*0.1
# Reset the max and min
plt.ylim(ymin, ymax)
ax.set(title=TIC + ' Sector ' + str(sector_no))
plt.show()

# Initialise the arrays before we start
coords = []
tran = []

# Click event to select all 6 parameter one after the other with right click.  
def onclick(event):
    global ix, iy, coords,tran
    ix, iy = event.xdata, event.ydata
    if event.button == 3:  # right click

        if len(coords) < 2: # Start and end
            coords.append((ix, iy))
            tran.append(plt.axvline(ix, color = 'blue', zorder = -1))  
        elif len(coords) < 4: # Start and end of floor
            coords.append((ix, iy))
            tran.append(plt.axvline(ix, color = 'grey', linestyle = '-', zorder = -1))       
        elif len(coords) < 6: # Top and bottom
            coords.append((ix, iy))
            tran.append(plt.axhline(iy, color = 'Green', zorder = -1))
        elif len(coords) == 6:
            for line in tran:
                try: 
                    line.remove()
                except:
                    pass
            tran.clear()   
            coords.clear()

cid = fig.canvas.mpl_connect('button_press_event', onclick)  # Connect the event

# Create a reset button
button_reset = widgets.Button(description='Reset', button_style='warning')

out_op = widgets.Output()  # Output from the button actions

# Define reset button code
def on_reset_button_clicked(_):
    global coords, tran
    with out_op:
        clear_output()
        for line in tran:
            try: 
                line.remove()
            except:
                pass
        tran.clear()   
        coords.clear()
        
# link button and function together using a button's method
button_reset.on_click(on_reset_button_clicked)        

# Create a button to complete the process
button_op = widgets.Button(description='Done', button_style='primary')

# Define to 'done' button actions
def on_op_button_clicked(_):
    global coords
    global transit_depth,transit_start,transit_end,transit_length,transit_mid,floor_length
    with out_op:
        clear_output()
        if len(coords) < 6:
            print("Select the six parameters before continuing")
        else:
            # Extract the co-ordinates for the transit dimensions
            transit_top = coords[4][1] 
            transit_bottom = coords[5][1] 
            transit_depth = transit_top-transit_bottom
            if transit_depth < 0 : transit_depth *= -1  # In case we choose the wrong way round
            print("Depth = %6.4f" %transit_depth)
            transit_start = coords[0][0] 
            transit_end = coords[1][0] 
            transit_length = (transit_end-transit_start) * 24
            if transit_length < 0 : transit_length *= -1  # In case we choose the wrong way round
            transit_mid = transit_start+(transit_end-transit_start)/2
            print('Mid-point = %6.4f' %transit_mid)
            print('Duration = %6.4f hours' %transit_length)
            floor_start = coords[2][0] 
            floor_end = coords[3][0] 
            floor_length = (floor_end-floor_start) * 24
            if floor_length < 0 : floor_length *= -1  # In case we choose the wrong way round
            print("Floor = %6.4f hours" %floor_length)   
            
# link button and function together using a button's method
button_op.on_click(on_op_button_clicked)    

# Create and display a display box containing the buttons and the output region
boxop = widgets.VBox([button_reset,button_op,out_op ])
boxop

# Planet Calculations

In [ ]:
import star_data as star
#import importlib
#importlib.reload(star)
GAIA_ID = TIC_rad = TIC_erad = TIC_mass = TIC_emass = TIC_Teff = TIC_eTeff = TIC_lum = TIC_Vmag = TIC_dist = TIC_mind = TIC_maxd = TIC_ra = TIC_dec = float('nan')
GAIA_ID, TIC_rad, TIC_erad, TIC_mass, TIC_emass, TIC_Teff, TIC_eTeff, TIC_lum, TIC_Vmag, TIC_dist, TIC_mind, TIC_maxd, TIC_ra, TIC_dec = star.get_exofop_data(TIC_no)
R_star = float(TIC_rad)
M_star = float(TIC_mass)
star.get_simbad_and_EB_data(TIC_no)

try:
    gaia_rad = gaia_rad_lower = gaia_rad_upper = gaia_rad_sigma = gaia_teff = gaia_teff_lower = gaia_teff_upper = gaia_teff_sigma = gaia_lum = gaia_lum_lower = gaia_lum_upper = gaia_mass = gaia_mass_lower = gaia_mass_upper = gaia_mass_sigma = float('nan')
    gaia_rad, gaia_rad_lower, gaia_rad_upper, gaia_rad_sigma, gaia_teff, gaia_teff_lower, gaia_teff_upper, gaia_teff_sigma, gaia_lum, gaia_lum_lower, gaia_lum_upper, gaia_mass, gaia_mass_lower, gaia_mass_upper, gaia_mass_sigma = star.get_gaia_data(GAIA_ID)
    if not(np.isnan(gaia_rad)):
        R_star= float(gaia_rad)
    if not(np.isnan(gaia_mass)):    
        M_star = float(gaia_mass)
except:
    print("Unable to get all GAIA data")

In [ ]:
# ============= Radius relative to sol ========================
#R_star = float(TIC_rad)
#R_star = float(gaia_rad)
# =============================================================
Rs = R_star * u.Rsun
r_pl_solar_radius = np.sqrt(transit_depth) * Rs
Rs_km = Rs.to(u.kilometer)
r_pl_solar_radius

In [ ]:
r_pl_earth_radius = r_pl_solar_radius.to(u.Rearth)
r_pl_earth_radius

In [ ]:
r_pl_jup_radius = r_pl_solar_radius.to(u.Rjupiter)
r_pl_jup_radius

In [ ]:
r_pl_km_radius = r_pl_solar_radius.to(u.kilometer)
r_pl_km_radius

In [ ]:
# ============ Mass of star relative to sol ==================
#M_star = float(TIC_mass)
#M_star = float(gaia_mass)
#M_star = 1
# ============================================================
transit_length_days = transit_length/24
floor_length_days = floor_length/24

# Impact Parameter
import math
sqdf = math.sqrt(transit_depth)
print(sqdf)
ts = (floor_length_days**2)/(transit_length_days**2)
print(ts)
#bs = math.sqrt((((1-sqdf)*(1-sqdf))-(ts*(1+sqdf)*(1+sqdf)))/(1.0-ts))
bs = math.sqrt((((1-sqdf)**2)-(ts*(1+sqdf)**2))/(1.0-ts))
print(bs)

# Tc
tc = transit_length/math.sqrt(1-(bs**2))

In [ ]:
import math
from astropy import units as u
Ms = M_star * u.Msun
Ms_g = Ms.to(u.gram)
print(Ms_g)
Rs = R_star * u.Rsun
Rs_cm = Rs.to(u.centimeter)
print(Rs_cm)
rho = Ms_g.value/((4/3)*math.pi*math.pow(Rs_cm.value,3))
print(rho)

In [ ]:
print ('\033[1mSummary\033[0m: ' + TIC)    
print ('T0 = %8.4f BJD' %transit_mid)
print ('Transit Depth = %6.4f' %(transit_depth))
print ('Transit Length = %6.4f hours' %(transit_length))
print ('Floor Length = %6.4f hours' %(floor_length))
print ('Star Radius = ' + f"{R_star:0.04f} Rsol")
print ('Star Mass = ' + f"{M_star:0.04f} Msol")
print ('Star_Density = %6.4f g/cm^3' %(rho))
print ('Vmag = %7.4f  GAIA = %d' %(TIC_Vmag, GAIA_ID))
print ('Planet Radius = ' + f"{r_pl_jup_radius:0.02f}" + "   " + f"{r_pl_earth_radius:0.02f}") 
print ('Distance = %8.4f (-%0.4f +%0.4f)pc' %(TIC_dist, TIC_mind, TIC_maxd))
print ('Impact Parameter = %5.4f' %bs)
print ('Transit Central (est.) = %6.4f hours' %tc)

# We have no period, this estimates it based on the transit duration but could be wildly inaccurate

# Separation
tcm = (tc/(13 * R_star))*math.sqrt(M_star)
au = tcm ** 2
au_km = (au * u.AU).to(u.kilometer)
i = math.degrees(math.acos((bs*Rs_km)/au_km))
period = math.sqrt((au*au*au)/M_star)*365

# A/r*
ar = (2 * period * math.pow(transit_depth,0.25))/(math.pi * math.sqrt((transit_length_days**2)-(floor_length_days**2)))

print ('A/r* = %6.4f' %ar)
print ('Period = %7.4f days' %(period))
print ('Separation = %6.4f AU' %au)
print ('i = %6.4f deg' %i)

# Fold the curve to look for possible periods

In [ ]:
#============================Change the line below to select the data required=====================
lc_select = available_data_select[0:9]
#==================================================================================================
lc_coll = lc_select.download_all().stitch()
print ("Done")

In [ ]:
# interactive finding the orbital period
# NOTE once you've run the below line of code, you will no longer be able to zoom in on the lightcurve! 
# You'll have to restart the notebook to be able to zoom in on the plots (even the ones earlier in this notebook) again! 

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

def plot_phase_folded_color(period, xrange=1):

    # phase fold the light curve
    lc_phased = lc_coll.fold(period = period, epoch_time = transit_time)
    
    # plot the binned and unbinned phase folded lightcurve on the same figure
    %matplotlib inline 
    fig, ax = plt.subplots(figsize = (30,12))
    
    plt.scatter(lc_phased.time.value, lc_phased.flux.value, c = lc_phased.time_original.value, s = 4, marker = 'o', cmap = 'spring')

    # if you prefer all the point to have the same color, delete the line above and get rid of the hastag in fron of the line below
    #lc_phased.plot(ax = ax, marker = '.', linewidth = 0, color = 'black', alpha = 0.2, markersize = 3)
    
    plt.xlabel("Phase")
    plt.ylabel("Normalized flux")
    plt.ylim(ymin, ymax)
    xlims = xrange
    plt.xlim(-xlims,xlims)
    #plt.ylim(0.995, 1.01)
    plt.show()

In [ ]:
# interactive widget - read above for what to change in this line of code :)
transit_time = (transit_end+transit_start)/2
#=========Set Starting Period, comment out to use calculated value ===============
starting_period=41
#=========Set X range starting value =============================================
xrange_start = 20
#=================================================================================
interact(plot_phase_folded_color, period = widgets.FloatSlider(value=starting_period,min =1,max=200,step=1,description='period:', readout_format='.5f'), xrange = widgets.FloatSlider(value=xrange_start,min =0,max=100,step=1,description='xrange:'))

# Export files for Pyaneti

In [ ]:
try:
    print("Exofop data")
    print("Radius = {}".format(*TIC_rad))
    print("Mass = {}".format(*TIC_mass))
    print("Teff = {}".format(*TIC_Teff))
    print("GAIA data")
    print("Radius = {}".format(*gaia_rad))
    print("Mass = {}".format(*gaia_mass))
    print("Teff = {}".format(*gaia_Teff))
except:
    pass

In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output
chk_exofop = widgets.Checkbox(
           value=False,
           description='Use exofop',)
menu_method = widgets.Dropdown(
       options=['DETREND', 'CUTOUT'],
       value='CUTOUT',
       description='Method:')
chk_density = widgets.Checkbox(
           value=False,
           description='Fit density:',)
suffix = widgets.Text(
       value='_single_fit',
       description='Suffix:', )
pmin = widgets.Text(
       value='15',
       description='Min P:', )
pmax = widgets.Text(
       value='200',
       description='Max P:', )

button_set = widgets.Button(description='Set Options', button_style='primary')
out_set = widgets.Output()
def on_set_button_clicked(_):
    global use_exofop
    global data_method
    global fit_density
    global model_suffix
    global min_p, max_p
    with out_set:
        clear_output()
        use_exofop = chk_exofop.value
        data_method = menu_method.value
        fit_density = chk_density.value
        model_suffix = suffix.value
        min_p = int(pmin.value)
        max_p = int(pmax.value)
        if use_exofop == True:
            print("Using exofop data")
        else:
            print("Using GAIA data")
        print("Data method: {}".format(data_method))
        if fit_density == True:
            print("Fitting density")
        else:
            print("Fitting semi-major axis")        
        print("File suffix: {}".format(model_suffix))
        print("Period range {} to {}".format(min_p,max_p))
        
# linking button and function together using a button's method
button_set.on_click(on_set_button_clicked)    

box4 = widgets.VBox([chk_exofop,menu_method,chk_density,suffix,pmin,pmax,button_set,out_set])
box4

In [ ]:
import os
import csv
import shutil

if use_exofop:
    pya_rad = TIC_rad
    pya_erad = TIC_erad
    pya_mass = TIC_mass
    pya_emass = TIC_emass
    pya_Teff = TIC_Teff
    pya_eTeff = TIC_eTeff
    radius_source = "Exofop"
    mass_source = "Exofop"
    Teff_source = "Exofop"
else:
    #GAIA data
    pya_rad = gaia_rad
    pya_erad = gaia_rad_sigma
    radius_source = "GAIA"
    if np.isnan(gaia_mass):
        pya_mass = TIC_mass
        mass_source = "Exofop"
    else:
        pya_mass = gaia_mass
        mass_source = "GAIA"
    if np.isnan(gaia_mass_sigma):
        pya_emass = TIC_emass
    else:
        pya_emass = gaia_mass_sigma
    if np.isnan(gaia_teff):
        pya_Teff = TIC_Teff
        Teff_source = "Exofop"
    else:
        pya_Teff = gaia_teff
        Teff_source = "GAIA"
    if np.isnan(gaia_teff_sigma):
        pya_eTeff = TIC_eTeff
    else:
        pya_eTeff = gaia_teff_sigma
    data_source = "GAIA"
    
if fit_density:
    min_a = rho
    max_a = rho*0.15
    is_single_transit = "False"
    sample_stellar_density = "True"
    fit_a = "g"
    fit_a_comment = "We fit a with gaussian priors (given by the stellar parameters)"
else:
    min_a = 0.001
    max_a = 200
    is_single_transit = "True"
    sample_stellar_density = "False"
    fit_a = "u"
    fit_a_comment = "We fit the scaled semi-major axis"    

# Pyaneti files

#pyaneti_home_dir = r".."  
pyaneti_home_dir = "/home/ian/pyaneti"
inpy_dir = pyaneti_home_dir + "/inpy/"
outpy_dir = pyaneti_home_dir + "/outpy/"
pyaneti_data_filename = '{}_data.txt'.format(TIC_no)
pyaneti_data_file = inpy_dir + '{}/'.format(TIC_no) + pyaneti_data_filename  
input_fit_filename = "input_fit.py"
pyaneti_input_file = inpy_dir + '{}/{}'.format(TIC_no,input_fit_filename)
pyaneti_command = 'python pyaneti.py {}'.format(TIC_no)
target_out_dir = outpy_dir + '{}_out'.format(TIC_no)
target_in_dir = inpy_dir + '{}'.format(TIC_no)
pyaneti_full_data_file = target_out_dir + '/full_' + pyaneti_data_filename 

#remove any existing files first
try:
    shutil.rmtree(target_out_dir)
except:
    pass
try:
    shutil.rmtree(target_in_dir)
except:
    pass

# Create directory to store the files
path = inpy_dir + TIC_no
try:
    os.mkdir(path)
except OSError:
    pass   # Assume its already there

path = target_out_dir
try:
    os.mkdir(path)
except OSError:
    pass   # Assume its already there

In [ ]:
import numpy as np
import pandas as pd
import astropy.io.fits as pf

def detrend_pyaneti(tic, transit_time_list, alltime, allflux, allflux_err, lim_window, lim, poly_n, save = True):
    
    
    combined_corr_time = []
    combined_corr_flux = []
    combined_corr_err  = []
    
    for transit_time in transit_time_list:
        try:

            mask = (np.array(alltime) > transit_time-lim_window) & (np.array(alltime) < transit_time+lim_window)

            masked_time= np.array(alltime)[mask]
            masked_flux = np.array(allflux)[mask]
            masked_flux_err = np.array(allflux_err)[mask]

            np.nan_to_num(masked_time,False,nan=0.0)  # Make dure there are no Nans as it breaks polyfit
            np.nan_to_num(masked_flux,False,nan=1.0)

            z = np.polyfit(masked_time, masked_flux, 1)

            p = np.poly1d(z)

            xp = np.linspace(np.nanmin(masked_time), np.nanmax(masked_time), 100)

            finite = np.isfinite(masked_time) & np.isfinite(masked_flux)
            
            x = masked_time[finite]
            y = masked_flux[finite]
            y_err = masked_flux_err[finite]
                      
            #plt.show()

            oot = (x > (transit_time-lim)) & (x < (transit_time+lim))
            
            x_oot = x[~oot]
            y_oot = y[~oot]
            
            # Detrend with a 2d order polynomial
            
            model = np.polyfit(x_oot, y_oot, poly_n)
            predicted = np.polyval(model, x)
            
            fig, axes = plt.subplots(nrows=2, sharex=True, figsize=(10,10))
            
            axes[0].plot(x, y, '#c44e52', marker='o', lw =0)
            axes[0].plot(x, predicted, 'k-')
            axes[0].set(title='Original Data and 2nd Order Polynomial Trend')
            axes[0].axvline(transit_time-lim)
            axes[0].axvline(transit_time+lim)
                            
            axes[1].plot(x, y - predicted,  '#c44e52', marker='o', lw =0)
            axes[1].set(title='Detrended Residual')
            
            combined_corr_time.append(x)
            combined_corr_flux.append((y - predicted) + 1)
            combined_corr_err.append(y_err)
            
            plt.show()
        except Exception as e: print(e)
            #print ("{} not in the data set".format(transit_time))

    # Include the first and last data points so the model extends to the full time range
    combined_corr_time.insert(0,alltime[0])
    combined_corr_flux.insert(0,allflux[0])
    combined_corr_err.insert(0,allflux_err[0])
    combined_corr_time.append(alltime[len(alltime)-1])  
    combined_corr_flux.append(allflux[len(allflux)-1])
    combined_corr_err.append(allflux_err[len(allflux_err)-1])            
            
    combined_corr_time = np.hstack(combined_corr_time)
    combined_corr_flux = np.hstack(combined_corr_flux)
    combined_corr_err  = np.hstack(combined_corr_err)
    
    finite_mask = np.isfinite(combined_corr_time) & np.isfinite(combined_corr_flux) & np.isfinite(combined_corr_err)
    
    combined_corr_time = combined_corr_time[finite_mask]
    combined_corr_flux = combined_corr_flux[finite_mask]
    combined_corr_err  = combined_corr_err[finite_mask]
    fig, axes = plt.subplots(figsize=(20,7))
    axes.plot(combined_corr_time,combined_corr_flux, marker='o', lw =0)
    plt.show()
    if save == True:
        np.savetxt(pyaneti_data_file, np.array([combined_corr_time, combined_corr_flux, combined_corr_err]).T)
        print("Created detrended data file: " + pyaneti_data_file)

In [ ]:
if data_method == "CUTOUT":  # Option to export just cutouts round the transit (no detrending)
    #CUTOUT
    # Data file - just the 2 days surrounding the dip (See below if full data file wanted - but it slows pyaneti down!)
    import csv

    # Export file for pyaneti
    with open(pyaneti_data_file, "w", newline='') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(['#time', 'flux', 'flux_err'])

    alltime = lc_collection.remove_nans().time.btjd
    allflux = lc_collection.remove_nans().flux
    allfluxerr = lc_collection.remove_nans().flux_err

    transit = (transit_end+transit_start)/2

    pyaneti_mask = (alltime > (transit - 1)) * (alltime < (transit + 1))
    
    # Include the first and last data points so the model extends to the full time range
    pyaneti_mask[0] = True
    pyaneti_mask[len(alltime)-1] = True    

    with open(pyaneti_data_file, "a", newline='') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(zip(alltime[pyaneti_mask],allflux[pyaneti_mask],allfluxerr[pyaneti_mask]))  # Comment out for full data
        #writer.writerows(zip(alltime,allflux,allfluxerr))  # Uncomment for full data

    f.close()
    fig, axes = plt.subplots(figsize=(20,7))
    axes.plot(alltime[pyaneti_mask],allflux[pyaneti_mask], marker='o', lw =0)
    plt.show()
    print("Created " + pyaneti_data_file)
elif data_method == "DETREND":  # Option to export detrended LC
    # DETREND    
    transit = (transit_end+transit_start)/2    
    lc_detrend = lc_collection.normalize()

    # Extract the time and fluxes
    alltime = np.array(lc_detrend.remove_nans().time.btjd)
    allflux = np.array(lc_detrend.remove_nans().flux.value)
    allflux_err = np.array(lc_detrend.remove_nans().flux_err.value)

    # to get a good fit you need to adjust these parameters

    # the size of the cut-off window - ideally you want this to be a couple of times the transit duration
    cutoutwindow_width = (transit_length/24)*3

    # this is to mask out the transit event (the stellar trend will be fit to the data excluding this data)
    # this widnow is marked on by the blue vertical lines
    transit_mask = transit_length/24

    # this is the order of the polynomial - don't go to high! I recomment no higher than 3. 
    polynomial_order = 4
    transit_time = []
    transit_time.append(transit)
    detrend_pyaneti(TIC_no, transit_time, alltime, allflux, allflux_err,cutoutwindow_width, transit_mask, polynomial_order, save = True)

    # the second panel shows the detrended LC - check that it looks okay. The data is automatically saved and ready for pyaneti.    
else:
    print ("Select an data method")
    
# Export full data for plotting later

lc_norm = lc_collection.normalize()
# Export file for pyaneti
with open(pyaneti_full_data_file, "w", newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(['#time', 'flux', 'flux_err'])

alltime = lc_norm.remove_nans().time.btjd
allflux = lc_norm.remove_nans().flux
allfluxerr = lc_norm.remove_nans().flux_err

with open(pyaneti_full_data_file, "a", newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerows(zip(alltime,allflux,allfluxerr))  # Uncomment for full data

f.close()
#fig, axes = plt.subplots(figsize=(20,7))
#axes.plot(alltime,allflux, marker='o', lw =0)
#plt.show()
print("Created full data file: " + pyaneti_full_data_file)

In [ ]:
#print(lc_collection['flux_err'])

In [ ]:
# Use this if flux_errs are NaNs in some of the data
#lc_collection['flux_err'] = [0.0003 if math.isnan(x) else x for x in lc_collection['flux_err']]
#lc_collection['flux_err'] = [0.0003 if x<0.0003 else x for x in lc_collection['flux_err']]
#print(lc_collection['flux_err'])

In [ ]:
# Pyaneti input data file
template = """#Input file for pyaneti
#fname_tr contains the transit data
fname_tr = ['{pyaneti_data_filename}']
'''
TIC {TIC_no}
Sectors {sectors}
Light Curve {data_method}
'''
#MCMC controls
#the thin factor for the chains
thin_factor = 10
#The number of iterations to be taken into account
#The TOTAL number of iterations for the burn-in phase is thin_factor*niter
niter       = 500
#Number of independent Markov chains for the ensemble sampler
nchains     = 100

#Choose the method that we want to use
# mcmc -> runs the mcmc fit program
# plot -> this option create the plots only if a previus run was done
method = 'mcmc'
#method = 'plot'

is_single_transit = {is_single_transit}

#If you want a plot with the seaborn library, is_seaborn_plot has to be True
is_seaborn_plot = False
plot_binned_data = True

#Define the star parameters to calculate the planet parameters
mstar_mean  = {pya_mass:0.04f}  # {mass_source}
mstar_sigma = {pya_emass:0.04f}
rstar_mean  = {pya_rad:0.04f}  # {radius_source}
rstar_sigma = {pya_erad:0.04f}
tstar_mean  = {pya_Teff:0.04f}  # {Teff_source}
tstar_sigma = {pya_eTeff:0.04f}

#What units do you prefer for your planet parameters?
# earth, jupiter or solar
unit_mass = 'earth'

#If we want posterior, correlation and/or chain plots these options have to be set True
is_plot_posterior    = True
is_plot_correlations = True
is_plot_chains       = False

#Are we setting gaussian priors on the semi-major axis based on the stellar parameters?
a_from_kepler = [True]

#We want to fit transit and RV 
#For a pure RV fit, fit_tr has to be False
#For a pure TR fit, fit_rv has to be False
#For multi-planet fits fit_rv and fit_tr have the form [True,True,False,...]
#one element for each planet.
fit_rv = [False]
fit_tr = [True]

#is_ew controls the parametrization sqrt(e)sin w and sqrt(e) cos w
#if True we fit for the parametrization parameters
#if False we fit for e and w
#Default is True
is_ew = True

# Change the next line to true if stellar density known and set priors for a min and max around known value
sample_stellar_density = {sample_stellar_density}

#Prior section
# f -> fixed value
# u -> Uniform priors
# g -> Gaussian priors
fit_t0 = ['u']   #We fit for t0 with uniform priors
fit_P  = ['u']   #We fit for P with uniform priors
fit_ew1= ['f']   #We fix sqrt(e) sin w, it works only if is_ew = True  - Change to u to model eccentricity
fit_ew2= ['f']   #We fix sqrt(e) cos w, it works only if is_ew = True  - Change to u to model eccentricity
fit_e  = ['f']   #We fix e, it works only if is_ew = False
fit_w  = ['f']   #We fix w, it works only if is_ew = False
fit_b  = ['u']   #We fit the impact factor
fit_a  = ['{fit_a}']   #{fit_a_comment}
fit_rp = ['u']   #We fit rp with uniform priors
fit_k  = ['u']   #We fit k with uniform priors
fit_v0 = 'u'     #We fit systemc velocities with uniform priors
fit_q1 = 'u'     #We fit q1 with gaussian priors
fit_q2 = 'u'     #We fit q2 with gaussian priors

#Prior ranges for a parameter A
#if 'f' is selected for the parameter A, A is fixed to the one given by min_A
#if 'u' is selected for the parameter A, sets uniform priors between min_A and max_A
#if 'g' is selected for the parameter A, sets gaussian priors with mean min_A and standard deviation max_A

min_t0  = [{transit_start:0.04f}] 
max_t0  = [{transit_end:0.04f}]  
min_P   = [{min_p}]
max_P   = [{max_p}]
min_ew1 = [0.0]  # Change to -1 if modelling ew
min_ew2 = [0.0]  # Change to -1 if modelling ew
max_ew1 = [1.0]
max_ew2 = [1.0]
min_a   = [{min_a}]
max_a   = [{max_a}]
min_b   = [0.0]
max_b   = [1.15]
min_k   = [0.0]
max_k   = [0.001]
min_rp  = [{prad_min:0.04f}]
max_rp  = [{prad_max:0.04f}]
min_q1  = 0.0
max_q1  = 1.0
min_q2  = 0.0
max_q2  = 1.0

""" 
context = {
"pyaneti_data_filename":pyaneti_data_filename,  
"TIC_no":TIC_no,
"sectors":sectors,
"data_method":data_method,
"radius_source":radius_source,
"mass_source":mass_source,
"Teff_source":Teff_source,
"pya_mass":float(pya_mass),
"pya_emass":float(pya_emass), 
"pya_rad":float(pya_rad),     
"pya_erad":float(pya_erad),    
"pya_Teff":float(pya_Teff),     
"pya_eTeff":float(pya_eTeff),
"is_single_transit":is_single_transit,
"sample_stellar_density":sample_stellar_density,
"fit_a":fit_a,
"fit_a_comment":fit_a_comment,
"min_p":min_p,
"max_p":max_p,
"min_a":min_a,
"max_a":max_a,
"transit_start":transit_start,
"transit_end":transit_end,
"prad_min":r_pl_solar_radius.value*0.2,
"prad_max":r_pl_solar_radius.value*2
} 

with open(pyaneti_input_file, "w", newline='') as f:
    #writer = csv.writer(f, delimiter='\t')
    f.write(template.format(**context))
    
f.close()
print("Created " + pyaneti_input_file)


In [ ]:
!gedit $pyaneti_input_file

In [ ]:
!echo Running Pyaneti
!cd $pyaneti_home_dir ; $pyaneti_command

In [ ]:
import pyaneti_utils as py
py.display_results(TIC_no, target_out_dir)

# Publish in PHT Shared

In [ ]:
import os
import shutil
from pathlib import Path
from distutils.dir_util import copy_tree

publish_dir = '/mnt/g/Google Drive/Astrophysics/PHT Shared/{}{}/'.format(TIC_no,model_suffix)

# Save the params `.dat` as `.txt` so that it can be easily viewed on Google Drive.
file_params = Path(target_out_dir, f"{TIC_no}_params.dat")
file_params_txt = Path(target_out_dir, f"{TIC_no}_params.txt")
shutil.copyfile(file_params, file_params_txt)

# Copy `input_fit.py` and data file to output directory so that it can be easily shared (on Google Drive).
destination = Path(target_out_dir, input_fit_filename)
shutil.copyfile(pyaneti_input_file, destination)
destination = Path(target_out_dir, pyaneti_data_filename)
shutil.copyfile(pyaneti_data_file, destination)

try:
    shutil.rmtree(publish_dir)
except:
    pass
try:
    os.mkdir(publish_dir)
except:
    pass

# Publish to PHT Shared and rename the model directories to preserve them
copy_tree(target_out_dir, publish_dir)
try:
    shutil.rmtree(target_out_dir + model_suffix)
except:
    pass
shutil.move(target_out_dir, target_out_dir + model_suffix)
try:
    shutil.rmtree(target_in_dir + model_suffix)
except:
    pass
shutil.move(target_in_dir, target_in_dir + model_suffix)

# Plot against known exoplanets

In [ ]:
# Check NASA Exoplanet Archive and retrieve planets with similar period and radius (within tolerance of each)
import astropy.units as u
from astropy.time import Time
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
import numpy as np

period_c =  float(period)
radius_c = r_pl_jup_radius.value

tolerance = 0.25

query_string = "(pl_orbper > %0.3f and pl_orbper < %0.3f) and (pl_radj > %0.3f and pl_radj < %0.3f)" %(period_c-(period_c*tolerance), period_c+(period_c*tolerance), radius_c-(radius_c*tolerance), radius_c+(radius_c*tolerance))
print(query_string)
planet_data = NasaExoplanetArchive.query_criteria(table="pscomppars", where=query_string, order="hostname")
if len(planet_data) > 0:
    planets = np.array(planet_data)  #extract table data into an array
    planets_df = pd.DataFrame(planets, columns=["pl_name","disc_year","pl_orbper","pl_trandur","pl_radj","pl_rade","st_rad","st_teff"])  #create a pandas data frame from the array and the headers
    print(planets_df)
else:
    print ("No Planets found")

In [ ]:
per=np.array(planet_data["pl_orbper"])
rad=np.array(planet_data["pl_radj"])
st_rad=np.array(planet_data["st_rad"])
teff=np.array(planet_data['st_teff'])

fig_plan, ax_plan = plt.subplots(figsize=(10, 10))
sc = plt.scatter(per, rad, s=st_rad*50, c = np.log(teff), cmap = 'RdYlBu', alpha = 0.9, zorder = -1, edgecolors = 'white', linewidths = 0.1)
sc.set_clim(np.log(2500), np.log(20000))
sc_target = plt.scatter(period_c,radius_c,s=R_star*50, c = np.log(TIC_Teff), cmap = 'RdYlBu', alpha = 0.8, zorder = -1, edgecolors = 'black', linewidths = 0.5)
sc_target.set_clim(np.log(2500), np.log(20000))
for i, txt in enumerate(planet_data["pl_name"]):
    ax_plan.annotate(txt, (per[i], rad[i]),color='grey',fontsize=10,textcoords="offset points",xytext=(4,3))  # Test is offest from the point by the xytext value
ax_plan.annotate(TIC, (period_c,radius_c),color='grey',fontsize=10,textcoords="offset points",xytext=(4,3))

plt.xlabel('Period (d)')
plt.ylabel('Radius (Rjup)')

# Create star chart round a target

In [ ]:
import star_data as star
TIC_Teff, TIC_lum, TIC_rad = star.plot_star_chart(TIC_no)

# HR Diagram

In [ ]:
import star_data as star
star.plot_HR(TIC_no, TIC_Teff, TIC_lum, TIC_rad, TOI = False)

# Old Calc actions - no longer used

In [ ]:
# Click event to select the TOP and BOTTOM of the Transit 
def onclick(event):
    global ix, iy
    ix, iy = event.xdata, event.ydata
    if event.button == 3:  # Only act on a right click
        global coords

        if len(coords) == 0:  # remove any existing markers if this is the first click
            try: 
                tran[0].remove()
            except:
                pass
            try: 
                tran[1].remove()
            except:
                pass
            tran.clear()

        coords.append((ix, iy))
        tran.append(plt.axhline(iy, color = 'orange', zorder = -1))

        if len(coords) == 2:
            fig.canvas.mpl_disconnect(cid)
            
# Click event to select the START and END of the Transit 
def onclick1(event):
    global ix, iy
    ix, iy = event.xdata, event.ydata
    if event.button == 3:  # Only act on a right click
        global coords

        if len(coords) == 0:  # remove any existing markers if this is the first click
            try: 
                tran[0].remove()
            except:
                pass
            try: 
                tran[1].remove()
            except:
                pass
            tran.clear()

        coords.append((ix, iy))
        tran.append(plt.axvline(ix, color = 'blue', zorder = -1))

        if len(coords) == 2:
            fig.canvas.mpl_disconnect(cid)
            
# Click event to select the Floor START and END of the Transit 
def onclick2(event):
    global ix, iy
    ix, iy = event.xdata, event.ydata
    if event.button == 3:  # Only act on a right click
        global coords

        if len(coords) == 0:  # remove any existing markers if this is the first click
            try: 
                tran[0].remove()
            except:
                pass
            try: 
                tran[1].remove()
            except:
                pass
            tran.clear()

        coords.append((ix, iy))
        tran.append(plt.axvline(ix, color = 'grey', linestyle = '-', zorder = -1))

        if len(coords) == 2:
            fig.canvas.mpl_disconnect(cid)
            

In [ ]:
# Click on the TOP and BOTTOM, then run this = transit depth

transit_top = coords[0][1] 
transit_bottom = coords[1][1] 
coords = []
cid = fig.canvas.mpl_connect('button_press_event', onclick)  # Reconnect the event in case we change our mind

transit_depth = transit_top-transit_bottom
if transit_depth < 0 : transit_depth *= -1  # In case we choose the wrong way round
transit_depth

In [ ]:
# When happy with top/bottom, run this to initialise event to select the START and END of the Transit - run this first
fig.canvas.mpl_disconnect(cid)
tran.clear()

cid = fig.canvas.mpl_connect('button_press_event', onclick1)  # Connect the event

In [ ]:
# Click on the START and END then run this = transit length

transit_start = coords[0][0] 
transit_end = coords[1][0] 
coords = []
cid = fig.canvas.mpl_connect('button_press_event', onclick1)  # Reconnect the event in case we change our mind

transit_length = (transit_end-transit_start) * 24
if transit_length < 0 : transit_length *= -1  # In case we choose the wrong way round
transit_mid = transit_start+(transit_end-transit_start)/2
print('Mid-point = %6.4f' %transit_mid)
print('Duration = %6.4f' %transit_length)

In [ ]:
# When happy with start/end, run this to initialise event to select the START and END of the Floor - run this first
fig.canvas.mpl_disconnect(cid)
tran.clear()

cid = fig.canvas.mpl_connect('button_press_event', onclick2)  # Connect the event

In [ ]:
# Click on the START and END of the floor then run this = Floor length

floor_start = coords[0][0] 
floor_end = coords[1][0] 
coords = []
cid = fig.canvas.mpl_connect('button_press_event', onclick2)  # Reconnect the event in case we change our mind

floor_length = (floor_end-floor_start) * 24
if floor_length < 0 : floor_length *= -1  # In case we choose the wrong way round
floor_length